### Solutions for https://github.com/Automating-GIS-processes-2022/Exercise-3/blob/main/Exercise-3-Problem-3.ipynb

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd

### Problem 3: How many people live near shopping centers? (8 points)

#### a) Load the population grid data set and the buffer geometries

In [14]:
population_grid = gpd.read_file(
    (
        "https://kartta.hsy.fi/geoserver/wfs"
        "?service=wfs"
        "&version=2.0.0"
        "&request=GetFeature"
        "&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020"
        "&srsName=EPSG:3879"
    ),
)
population_grid.crs = "EPSG:3879"
population_grid = population_grid[["asukkaita", "geometry"]]
population_grid = population_grid.rename(columns={"asukkaita": "population"})
population_grid.head()

,population,geometry
0,5,"POLYGON ((25472499.995 6685998.998, 25472499.9..."
1,8,"POLYGON ((25472499.995 6684249.004, 25472499.9..."
2,5,"POLYGON ((25472499.995 6683999.005, 25472499.9..."
3,13,"POLYGON ((25472499.995 6682998.998, 25472499.9..."
4,5,"POLYGON ((25472749.993 6690249.003, 25472749.9..."


In [17]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
import pyproj

assert isinstance(population_grid, geopandas.GeoDataFrame)
assert population_grid.crs == pyproj.CRS("EPSG:3879")

In [16]:
shopping_centre_buffers = gpd.read_file("data/shopping_centres.gpkg", layer="buffers")
shopping_centre_buffers

,id,name,addr,address,geometry
0,0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","POLYGON ((25506098.602 6677662.109, 25506091.3..."
1,1,Forum,"Mannerheimintie 14–20, 00100 Helsinki, Finland","Salaattiasema, 14-20, Mannerheimintie, Keskust...","POLYGON ((25498073.542 6672878.360, 25498066.3..."
2,2,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland","Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...","POLYGON ((25486970.435 6672070.967, 25486963.2..."
3,3,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finland","Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...","POLYGON ((25490991.076 6678322.265, 25490983.8..."
4,4,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa, Finland","Stockmann, 3, Vantaanportinkatu, Vantaanportti...","POLYGON ((25499443.932 6686656.982, 25499436.7..."
5,5,REDI,"Hermannin rantatie 5, 00580 Helsinki, Finland","Oluthuone Haavi, 5, Hermannin rantatie, Verkko...","POLYGON ((25500369.737 6674838.689, 25500362.5..."
6,6,Tripla,"Fredikanterassi 1, 00520 Helsinki, Finland","Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...","POLYGON ((25497645.658 6676164.227, 25497638.4..."


In [18]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
assert isinstance(shopping_centre_buffers, geopandas.GeoDataFrame)
assert shopping_centre_buffers.geometry.geom_type.unique() == ["Polygon"]
assert shopping_centre_buffers.crs == pyproj.CRS("EPSG:3879")

In [21]:
shopping_centre_buffers.sindex.valid_query_predicates

{None,
 'contains',
 'contains_properly',
 'covered_by',
 'covers',
 'crosses',
 'intersects',
 'overlaps',
 'touches',
 'within'}

#### b) Carry out a spatial join between the `population_grid` and the `shopping_centre_buffers`

In [26]:
joined = shopping_centre_buffers.sjoin(population_grid, how="inner", predicate="contains")
joined

,id,name,addr,address,geometry,index_right,population
0,0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","POLYGON ((25506098.602 6677662.109, 25506091.3...",5232,63
0,0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","POLYGON ((25506098.602 6677662.109, 25506091.3...",5231,43
0,0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","POLYGON ((25506098.602 6677662.109, 25506091.3...",5230,80
0,0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","POLYGON ((25506098.602 6677662.109, 25506091.3...",5229,319
0,0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","POLYGON ((25506098.602 6677662.109, 25506091.3...",5344,202
...,...,...,...,...,...,...,...
6,6,Tripla,"Fredikanterassi 1, 00520 Helsinki, Finland","Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...","POLYGON ((25497645.658 6676164.227, 25497638.4...",3388,17
6,6,Tripla,"Fredikanterassi 1, 00520 Helsinki, Finland","Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...","POLYGON ((25497645.658 6676164.227, 25497638.4...",3235,162
6,6,Tripla,"Fredikanterassi 1, 00520 Helsinki, Finland","Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...","POLYGON ((25497645.658 6676164.227, 25497638.4...",3573,395
6,6,Tripla,"Fredikanterassi 1, 00520 Helsinki, Finland","Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...","POLYGON ((25497645.658 6676164.227, 25497638.4...",3572,963


#### c) Compute the population sum around shopping centres

In [34]:
for name, population in joined.groupby("name")["population"].sum().items():
    print(f"{population} people live within 1.5 km from {name}.")

57062 people live within 1.5 km from Forum.
26698 people live within 1.5 km from Iso-omena.
20889 people live within 1.5 km from Itis.
10907 people live within 1.5 km from Jumbo.
27164 people live within 1.5 km from REDI.
22329 people live within 1.5 km from Sello.
24336 people live within 1.5 km from Tripla.
